<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/Transformers1_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

In [ ]:
sentiment.model

In [ ]:
print(sentiment(["I like olympic games as it's very exciting."]))
print(sentiment(["I'm against to hold Olympic game in Tokyo in terms of preventing the covid19 to be spread."]))


In [ ]:
qa = pipeline('question-answering')

olympic_wiki_text = """
The 2020 Summer Olympics,[c] officially the Games of the XXXII Olympiad[d] and officially branded as Tokyo 2020,[e] was an international multi-sport event that was held from 23 July to 8 August 2021 in Tokyo, Japan, with some of the preliminary sporting events beginning on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina on 7 September 2013.[3]

Originally scheduled to take place from 24 July to 9 August 2020, the Tokyo Games were postponed until 2021 on 24 March 2020 as a result of the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (some previous editions had been cancelled but not rescheduled).[4][5] However, the Tokyo 2020 branding was retained for marketing purposes.[6] The events were largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the only Olympic Games to be held without official spectators.[f] As a consequence of the postponement and the additional challenges caused by the pandemic, the 2020 Games were the most costly ever, with a total expenditure of over $20 billion.[8]

The 2020 Games were the fourth Olympics to be held in Japan, following the 1964 Summer Olympics (Tokyo), the 1972 Winter Olympics (Sapporo), and the 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice.[g] The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Because of the one-year postponement, Tokyo 2020 is the only Olympic Games to have taken place in an odd-numbered year.[10]
"""

print(qa(question="What cause Tokyo Olympic postponed?", context=olympic_wiki_text))

In [ ]:
qa.model

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!pip install transformers
!pip install torchtext==0.15.2
!pip install portalocker==2.7.0
!pip install accelerate -U

In [ ]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

# 출력 결과를 고정하기 위해
import random
random.seed(6)

# train_iter를 리스트 타입으로 변경
train_list = list(train_iter)
test_list = list(test_iter)

# 각기 1000개씩 random sampling
train_list_small = random.sample(train_list, 1000)
test_list_small = random.sample(test_list, 1000)

# 각 변수에 담긴 인덱스 0의 내용 화면 출력
print(train_list_small[0])
print(test_list_small[0])

In [ ]:
# train_text, train_label 컨테이너 생성
# 아래 반복문에서 생성된 결과를 담는다.
train_texts = []
train_labels = []

#for 반복문
# train_list_small에 담긴 튜플쌍 원소를 변수명 label, test부여하여 순서대로 축출
for label, text in train_list_small:
  # IMDB 데이터의 기존 레이블 2를 1로 변경, 기존 레이블 1을 0으로 변경.
  train_labels.append(1 if label == 2 else 0)
  train_texts.append(text)

# test_list_small도 동일하게 처리
test_texts = []
test_labels = []
for label, text in test_list_small:
  test_labels.append(1 if label==2 else 0)
  test_texts.append(text)

# 각 변수에 담긴 첫번째 원소 출력
print(train_texts[0])
print(train_labels[0])
print(test_texts[0])
print(test_labels[0])

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2,random_state=3)

print(len(train_texts))
print(len(train_labels))
print(len(val_texts))
print(len(val_labels))

In [ ]:
# colab 런타임이 끊어지는 경우에만 재 실행.
# !pip install transformers

# distilbert-base-uncased 모델에서 tokenizer 불러오기
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# tokenizer execute
# truncation=True -> 모델의 default max_length를 넘는 입력부분은 더이상 받지않고 절단한다는 의미
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# 0번째 입력문의 5번째 토큰까지 input_ids출력
print(train_encodings["input_ids"][0][:5])

#위의 결과를 다시 디코딩해서 출력
print(tokenizer.decode(train_encodings["input_ids"][0][:5]))


In [ ]:
import torch

# Dataset 클래스를 상속하는 IMDB class정의
class IMdbDataset(torch.utils.data.Dataset):

  # contructor
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  #
  def __getitem__(self, idx):
    # self.encoding에 담긴 키(key)와 키값(value)를 items()로 축출
    # 이 값을 key와 val 변수에 담아 새로운 키와 키값(torch.tensor(val[idx]))을 갖는 딕셔너리 생성

    # 딕셔너리{"key1 : value1", ...} 형태를 가지는 데이터 구조.
    # val[idx]에 담긴 데이터를 torch.tensor()을 통해 텐셔화.
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items() }

    # tensor화
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  #
  def __len__(self):
    return len(self.labels)

train_dataset = IMdbDataset(train_encodings, train_labels)
val_dataset = IMdbDataset(val_encodings, val_labels)
test_dataset = IMdbDataset(test_encodings, test_labels)

for i in train_dataset:
  print(i)
  break
